In [1]:
%pip install --quiet pandas pingouin

Note: you may need to restart the kernel to use updated packages.


In [2]:
coordinates = ( 'attitude', 'introspection', 'sensitivity', 'temper' )
hourglass = {
    'enthusiasm'    : { 'sensitivity' : 3 },
    'eagerness'     : { 'sensitivity' : 2 },
    'responsiveness': { 'sensitivity' : 1 },
    'anxiety'       : { 'sensitivity' : -1 },
    'fear'          : { 'sensitivity' : -2 },
    'terror'        : { 'sensitivity' : -3 },
    'bliss'         : { 'temper' : 3 },
    'calmness'      : { 'temper' : 2 },
    'serenity'      : { 'temper' : 1 },
    'annoyance'     : { 'temper' : -1 },
    'anger'         : { 'temper' : -2 },
    'rage'          : { 'temper' : -3 },
    'delight'       : { 'attitude' : 3 },
    'pleasantness'  : { 'attitude' : 2 },
    'acceptance'    : { 'attitude' : 1 },
    'dislike'       : { 'attitude' : -1 },
    'disgust'       : { 'attitude' : -2 },
    'loathing'      : { 'attitude' : -3 },
    'ecstasy'       : { 'introspection' : 3 },
    'joy'           : { 'introspection' : 2 },
    'contentment'   : { 'introspection' : 1 },
    'melancholy'    : { 'introspection' : -1 },
    'sadness'       : { 'introspection' : -2 },
    'grief'         : { 'introspection' : -3 }
}

## Expert annotation to Pandas

In [3]:
import json
with open('experts.json') as f:
    annotations = json.load(f)

In [4]:
import pandas as pd
import random, statistics

def calculate_rating(array):
    return sum(array)  # linear combination
    # return sum(array) / len(array)  # Average
    # return statistics.median(array)  # Median

iconclasses = {}
raters = set()
ratings_dict = {}

for task in annotations:
    for ann in task['annotations']:
        item = f"{task['id']}"
        iconclasses[item] = task["iconclass"]
        rater = f"{ann['completed_by']}"
        raters.add(rater)
        for r in ann['result']:
            label = r['value']['polygonlabels'][0]
            rating = ratings_dict.setdefault(item, {}).setdefault(rater, {})
            if label in hourglass:
                for coord in hourglass[label]:
                    rating.setdefault(coord, []).append(hourglass[label][coord])

In [5]:
expert_ratings = []
r_iphigenia = []
r_polyxena = []
r_helenandparis = []

for idd,item in ratings_dict.items():
    if "94D132" in iconclasses[idd]: r_secondary = r_iphigenia
    elif "94H243" in iconclasses[idd]: r_secondary = r_polyxena
    else: r_secondary = r_helenandparis
    for rater in raters:
        appraisal = item[rater]
        for coord in coordinates:
            try:
                rating = calculate_rating(item[rater][coord])
            except:
                rating = 0.0
            o = { 'ID' : idd, 'Rater' : rater, coord: rating }
            expert_ratings.append(o)
            r_secondary.append(o)
expert_ratings

[{'ID': '171671643', 'Rater': '64261', 'attitude': 0.0},
 {'ID': '171671643', 'Rater': '64261', 'introspection': 0.0},
 {'ID': '171671643', 'Rater': '64261', 'sensitivity': 1},
 {'ID': '171671643', 'Rater': '64261', 'temper': 0.0},
 {'ID': '171671643', 'Rater': '70243', 'attitude': 0.0},
 {'ID': '171671643', 'Rater': '70243', 'introspection': 0.0},
 {'ID': '171671643', 'Rater': '70243', 'sensitivity': 4},
 {'ID': '171671643', 'Rater': '70243', 'temper': 0.0},
 {'ID': '171675026', 'Rater': '64261', 'attitude': 0.0},
 {'ID': '171675026', 'Rater': '64261', 'introspection': 0.0},
 {'ID': '171675026', 'Rater': '64261', 'sensitivity': 3},
 {'ID': '171675026', 'Rater': '64261', 'temper': 0.0},
 {'ID': '171675026', 'Rater': '70243', 'attitude': 0.0},
 {'ID': '171675026', 'Rater': '70243', 'introspection': 0.0},
 {'ID': '171675026', 'Rater': '70243', 'sensitivity': 6},
 {'ID': '171675026', 'Rater': '70243', 'temper': 2},
 {'ID': '171676295', 'Rater': '64261', 'attitude': 0.0},
 {'ID': '17167629

In [6]:
import pingouin as pg

### Sacrifices

In [7]:
df = pd.concat([pd.DataFrame(r_iphigenia), pd.DataFrame(r_polyxena)])
icc = pg.intraclass_corr(data=df, targets='ID', raters='Rater', ratings='sensitivity').round(3)
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,-0.055,0.896,8,9,0.556,"[-0.64, 0.59]"
1,ICC2,Single random raters,0.158,1.717,8,8,0.231,"[-0.19, 0.63]"
2,ICC3,Single fixed raters,0.264,1.717,8,8,0.231,"[-0.44, 0.77]"
3,ICC1k,Average raters absolute,-0.116,0.896,8,9,0.556,"[-3.58, 0.74]"
4,ICC2k,Average random raters,0.272,1.717,8,8,0.231,"[-0.47, 0.78]"
5,ICC3k,Average fixed raters,0.418,1.717,8,8,0.231,"[-1.58, 0.87]"


In [8]:
icc = pg.intraclass_corr(data=df, targets='ID', raters='Rater', ratings='temper').round(3)
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,-0.271,0.574,8,9,0.777,"[-0.75, 0.43]"
1,ICC2,Single random raters,-0.214,0.619,8,8,0.744,"[-0.7, 0.46]"
2,ICC3,Single fixed raters,-0.235,0.619,8,8,0.744,"[-0.75, 0.47]"
3,ICC1k,Average raters absolute,-0.744,0.574,8,9,0.777,"[-6.15, 0.6]"
4,ICC2k,Average random raters,-0.545,0.619,8,8,0.744,"[-4.66, 0.63]"
5,ICC3k,Average fixed raters,-0.615,0.619,8,8,0.744,"[-6.16, 0.64]"


In [9]:
icc = pg.intraclass_corr(data=df, targets='ID', raters='Rater', ratings='attitude').round(3)
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.529,3.25,8,9,0.049,"[-0.12, 0.87]"
1,ICC2,Single random raters,0.533,3.37,8,8,0.053,"[-0.09, 0.87]"
2,ICC3,Single fixed raters,0.542,3.37,8,8,0.053,"[-0.14, 0.87]"
3,ICC1k,Average raters absolute,0.692,3.25,8,9,0.049,"[-0.26, 0.93]"
4,ICC2k,Average random raters,0.696,3.37,8,8,0.053,"[-0.21, 0.93]"
5,ICC3k,Average fixed raters,0.703,3.37,8,8,0.053,"[-0.32, 0.93]"


In [10]:
icc = pg.intraclass_corr(data=df, targets='ID', raters='Rater', ratings='introspection').round(3)
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.579,3.753,8,9,0.033,"[-0.04, 0.88]"
1,ICC2,Single random raters,0.590,4.289,8,8,0.027,"[0.01, 0.89]"
2,ICC3,Single fixed raters,0.622,4.289,8,8,0.027,"[-0.02, 0.9]"
3,ICC1k,Average raters absolute,0.734,3.753,8,9,0.033,"[-0.09, 0.94]"
4,ICC2k,Average random raters,0.742,4.289,8,8,0.027,"[0.02, 0.94]"
5,ICC3k,Average fixed raters,0.767,4.289,8,8,0.027,"[-0.03, 0.95]"


### Helen and Paris

In [11]:
df = pd.DataFrame(r_helenandparis)
icc = pg.intraclass_corr(data=df, targets='ID', raters='Rater', ratings='sensitivity').round(3)
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.554,3.484,14,15,0.011,"[0.09, 0.82]"
1,ICC2,Single random raters,0.552,3.420,14,14,0.014,"[0.08, 0.82]"
2,ICC3,Single fixed raters,0.547,3.420,14,14,0.014,"[0.07, 0.82]"
3,ICC1k,Average raters absolute,0.713,3.484,14,15,0.011,"[0.17, 0.9]"
4,ICC2k,Average random raters,0.711,3.420,14,14,0.014,"[0.15, 0.9]"
5,ICC3k,Average fixed raters,0.708,3.420,14,14,0.014,"[0.13, 0.9]"


In [12]:
icc = pg.intraclass_corr(data=df, targets='ID', raters='Rater', ratings='temper').round(3)
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.643,4.600,14,15,0.003,"[0.23, 0.86]"
1,ICC2,Single random raters,0.638,4.295,14,14,0.005,"[0.19, 0.86]"
2,ICC3,Single fixed raters,0.622,4.295,14,14,0.005,"[0.18, 0.85]"
3,ICC1k,Average raters absolute,0.783,4.600,14,15,0.003,"[0.37, 0.93]"
4,ICC2k,Average random raters,0.779,4.295,14,14,0.005,"[0.32, 0.93]"
5,ICC3k,Average fixed raters,0.767,4.295,14,14,0.005,"[0.31, 0.92]"


In [13]:
icc = pg.intraclass_corr(data=df, targets='ID', raters='Rater', ratings='attitude').round(3)
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.097,1.216,14,15,0.355,"[-0.41, 0.56]"
1,ICC2,Single random raters,0.083,1.175,14,14,0.384,"[-0.45, 0.56]"
2,ICC3,Single fixed raters,0.080,1.175,14,14,0.384,"[-0.43, 0.56]"
3,ICC1k,Average raters absolute,0.177,1.216,14,15,0.355,"[-1.38, 0.72]"
4,ICC2k,Average random raters,0.153,1.175,14,14,0.384,"[-1.67, 0.72]"
5,ICC3k,Average fixed raters,0.149,1.175,14,14,0.384,"[-1.54, 0.71]"


In [14]:
icc = pg.intraclass_corr(data=df, targets='ID', raters='Rater', ratings='introspection').round(3)
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.639,4.547,14,15,0.003,"[0.22, 0.86]"
1,ICC2,Single random raters,0.637,4.365,14,14,0.005,"[0.2, 0.86]"
2,ICC3,Single fixed raters,0.627,4.365,14,14,0.005,"[0.19, 0.86]"
3,ICC1k,Average raters absolute,0.780,4.547,14,15,0.003,"[0.36, 0.93]"
4,ICC2k,Average random raters,0.778,4.365,14,14,0.005,"[0.34, 0.93]"
5,ICC3k,Average fixed raters,0.771,4.365,14,14,0.005,"[0.32, 0.92]"
